In [2]:
import torch 
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn 
import torch.optim as optim
import torch.nn.functional as F
import numpy as np

C:\Users\abhir\anaconda3\lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: '[WinError 127] The specified procedure could not be found'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [3]:
transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform= transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False)

Files already downloaded and verified
Files already downloaded and verified


In [71]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN,self).__init__()
        self.conv1 = nn.Conv2d(3,32,3)
        self.conv2 = nn.Conv2d(32,32,3)
        self.pool = nn.MaxPool2d(2,2)
        self.conv3 = nn.Conv2d(32,64,3)
        self.conv4 = nn.Conv2d(64,64,3, padding= 1)
        self.pool2 = nn.MaxPool2d(2,2)
        self.fc1 = nn.Linear(64*1*1,512)
        self.fc2 = nn.Linear(512,128)
        self.fc3 = nn.Linear(128,10)
    
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool2(F.relu(self.conv3(x)))
        x = self.pool2(F.relu(self.conv4(x)))
         
        #print(x.size(1), x.size(2), x.size(3))
        x = x.view(-1,64*1*1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

ner = CNN()

In [72]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(ner.parameters(), lr=0.01, momentum=0) 

In [75]:
batch_size = 256
learning_rate = 0.001
num_epochs = 40
# Training loop
for epoch in range(num_epochs):
    for i,(data, target) in enumerate(trainloader):  # Iterate over training data in batches
        # Forward pass
        output = ner(data)
        loss = criterion(output, target)
        _,predicted = torch.max(output[:len(target)],1)
        accuracy = (predicted == target).sum().item()/len(target)

        # Backward pass and optimizer step
        optimizer.zero_grad()  # Reset gradients
        loss.backward()
        optimizer.step()
        
    print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(trainloader)}], Loss: {loss.item():.4f}, Accuracy: {accuracy:.4f}')



Epoch [1/40], Step [782/782], Loss: 1.0643, Accuracy: 0.5625
Epoch [2/40], Step [782/782], Loss: 1.1893, Accuracy: 0.6875
Epoch [3/40], Step [782/782], Loss: 0.9280, Accuracy: 0.6250
Epoch [4/40], Step [782/782], Loss: 0.8646, Accuracy: 0.7500
Epoch [5/40], Step [782/782], Loss: 0.8032, Accuracy: 0.7500
Epoch [6/40], Step [782/782], Loss: 0.6382, Accuracy: 0.8750
Epoch [7/40], Step [782/782], Loss: 1.0699, Accuracy: 0.6875
Epoch [8/40], Step [782/782], Loss: 0.7000, Accuracy: 0.7500
Epoch [9/40], Step [782/782], Loss: 0.4810, Accuracy: 0.8125
Epoch [10/40], Step [782/782], Loss: 1.0660, Accuracy: 0.6250
Epoch [11/40], Step [782/782], Loss: 0.7302, Accuracy: 0.7500
Epoch [12/40], Step [782/782], Loss: 0.5530, Accuracy: 0.7500
Epoch [13/40], Step [782/782], Loss: 0.6490, Accuracy: 0.8125
Epoch [14/40], Step [782/782], Loss: 0.5633, Accuracy: 0.7500
Epoch [15/40], Step [782/782], Loss: 0.8116, Accuracy: 0.6875
Epoch [16/40], Step [782/782], Loss: 0.7055, Accuracy: 0.6250
Epoch [17/40], St

In [76]:
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in testloader:
        outputs = ner(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print(f'Accuracy of the network on the 10000 test images: {100 * correct / total:.2f}%')

print('Finished Training')

Accuracy of the network on the 10000 test images: 68.43%
Finished Training


# Using BatchNormalization
Adding batch normalization layers in the neural network model

# Using dropout 
adding dropout layer

# using Everything

In [77]:
class Dropout(nn.Module):
    def __init__(self):
        super(Dropout,self).__init__()
        self.conv1 = nn.Conv2d(3,32,3)
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32,32,3)
        self.bn2 = nn.BatchNorm2d(32)
        self.pool = nn.MaxPool2d(2,2)
        self.conv3 = nn.Conv2d(32,64,3)
        self.bn3 = nn.BatchNorm2d(64)
        self.conv4 = nn.Conv2d(64,64,3, padding= 1)
        self.bn4 = nn.BatchNorm2d(64)
        self.pool2 = nn.MaxPool2d(2,2)
        self.fc1 = nn.Linear(64*1*1,512)
        self.fc2 = nn.Linear(512,128)
        self.fc3 = nn.Linear(128,10)
        self.dropout = nn.Dropout(p=0.5)
    
    def forward(self, x):
        x = self.pool(F.relu(self.bn1(self.conv1(x))))
        x = self.pool(F.relu(self.bn2(self.conv2(x))))
        x = self.pool2(F.relu(self.bn3(self.conv3(x))))
        x = self.pool2(F.relu(self.bn4(self.conv4(x))))
         
        #print(x.size(1), x.size(2), x.size(3))
        x = x.view(-1,64*1*1)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.fc3(x)
        return x

drop = Dropout()

In [83]:
l2_lambda = 0.001
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(drop.parameters(), lr=0.01, momentum=0, weight_decay=l2_lambda)
batch_size = 256
lr = [0.1,0.01,0.001]
num_epochs = 10
best_accuracy = 0
# Training loop
for learning_rate in lr:
    print("Learning Rate : ", learning_rate)
    for epoch in range(num_epochs):
        for i,(data, target) in enumerate(trainloader):  # Iterate over training data in batches
            # Forward pass
            output = drop(data)
            loss = criterion(output, target)
            _,predicted = torch.max(output[:len(target)],1)
            accuracy = (predicted == target).sum().item()/len(target)

            # Backward pass and optimizer step
            optimizer.zero_grad()  # Reset gradients
            loss.backward()
            optimizer.step()
            
            if best_accuracy < accuracy:
                best_accuracy = accuracy

        print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(trainloader)}], Loss: {loss.item():.4f}, Accuracy: {accuracy:.4f}')
        
    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in testloader:
            outputs = drop(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        print(f'Accuracy of the network on the 10000 test images: {100 * correct / total:.2f}%')
    print('Finished Training')
print("Best Accuracy : ", best_accuracy)

Learning Rate :  0.1
Epoch [1/10], Step [782/782], Loss: 0.8926, Accuracy: 0.5000
Epoch [2/10], Step [782/782], Loss: 0.8110, Accuracy: 0.7500
Epoch [3/10], Step [782/782], Loss: 1.0699, Accuracy: 0.6250
Epoch [4/10], Step [782/782], Loss: 1.0075, Accuracy: 0.6250
Epoch [5/10], Step [782/782], Loss: 0.9673, Accuracy: 0.8125
Epoch [6/10], Step [782/782], Loss: 0.8055, Accuracy: 0.7500
Epoch [7/10], Step [782/782], Loss: 0.7628, Accuracy: 0.6875
Epoch [8/10], Step [782/782], Loss: 0.4392, Accuracy: 0.8125
Epoch [9/10], Step [782/782], Loss: 0.6588, Accuracy: 0.6875
Epoch [10/10], Step [782/782], Loss: 1.2137, Accuracy: 0.7500
Accuracy of the network on the 10000 test images: 73.29%
Finished Training
Learning Rate :  0.01
Epoch [1/10], Step [782/782], Loss: 1.4348, Accuracy: 0.6875
Epoch [2/10], Step [782/782], Loss: 0.6557, Accuracy: 0.8750
Epoch [3/10], Step [782/782], Loss: 0.8736, Accuracy: 0.7500
Epoch [4/10], Step [782/782], Loss: 0.5747, Accuracy: 0.8750
Epoch [5/10], Step [782/782

In [84]:
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in testloader:
        outputs = drop(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print(f'Accuracy of the network on the 10000 test images: {100 * correct / total:.2f}%')
print('Finished Training')

Accuracy of the network on the 10000 test images: 72.77%
Finished Training


# Resnet18

In [ ]:
import torchvision.models as models
# Define ResNet18 architecture
resnet18 = models.resnet18(pretrained=True)  # Load pre-trained ResNet18 architecture

# Modify the last layer to have 10 outputs for CIFAR-10
num_ftrs = resnet18.fc.in_features
resnet18.fc = nn.Linear(num_ftrs, 10)  # Modify the last fully connected layer

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(resnet18.parameters(), lr=0.001, momentum=0.9)

# Train the network
for epoch in range(10):  # Example: Train for 10 epochs
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        optimizer.zero_grad()

        outputs = resnet18(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 200 == 199:  # Print every 200 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 200))
            running_loss = 0.0

print('Finished Training')

# Evaluate the network
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = resnet18(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))